In [1]:
%pip install --quiet psutil py-cpuinfo pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psutil
import platform
from datetime import datetime
import cpuinfo
import socket
import uuid
import re
import pandas as pd  # Import pandas


def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor


# Your get_size function remains unchanged


def System_information():
    info = []  # List to hold information tuples

    # System Information
    uname = platform.uname()
    info.append(("System", uname.system))
    info.append(("Node Name", uname.node))
    info.append(("Release", uname.release))
    info.append(("Version", uname.version))
    info.append(("Machine", uname.machine))
    info.append(("Processor", uname.processor))
    info.append(("Processor Brand", cpuinfo.get_cpu_info()["brand_raw"]))

    # CPU Information
    info.append(("Physical cores", psutil.cpu_count(logical=False)))

    # Memory Information
    svmem = psutil.virtual_memory()
    info.append(("Total Memory", get_size(svmem.total)))

    # Disk Information
    partitions = psutil.disk_partitions()
    for partition in partitions:
        if partition.mountpoint in ["/", "/boot"]:
            try:
                partition_usage = psutil.disk_usage(partition.mountpoint)
                info.append(
                    (
                        f"Disk: {partition.device} Total Size",
                        get_size(partition_usage.total),
                    )
                )
            except PermissionError:
                # Skip the partition if permission error
                continue

    # Convert list of tuples to DataFrame
    df = pd.DataFrame(info, columns=["Item", "Value"])
    return df  # Return DataFrame instead of printing


if __name__ == "__main__":
    print(datetime.now().strftime("%y-%m-%d-%H-%M"))
    df = System_information()

    # Use the style property to set text alignment to left
    styled_df = df.style.set_properties(**{"text-align": "left"})

    # Set the table styles
    styled_df.set_table_styles([dict(selector="th", props=[("text-align", "left")])])

    # Print the sytled df
    display(styled_df)

24-03-12-21-38


,Item,Value
0,System,Darwin
1,Node Name,CQFYX0V9G3
2,Release,23.2.0
3,Version,Darwin Kernel Version 23.2.0: Wed Nov 15 21:53:18 PST 2023; root:xnu-10002.61.3~2/RELEASE_ARM64_T6000
4,Machine,arm64
5,Processor,arm
6,Processor Brand,Apple M1 Max
7,Physical cores,10
8,Total Memory,64.00GB
9,Disk: /dev/disk3s1s1 Total Size,926.35GB
